In [34]:
import numpy as np

class orbitElements:

    # t0 is in days, M is in kg
    # Takes in two options:
    # ONE:
    # elements = (a, i, e, LAN, omega, M0)
    # a = Semi-major axis (m)
    # i = Inclination (rad)
    # e = Eccentricity
    # LAN = Longitude of ascending node (rad)
    # omega = Argument of periapsis (rad)
    # M0 = Mean anomaly at t = 0 (radians)
    # TWO:
    # Position vector (m)
    # Velocity vector (m/s)


    def __init__(self, elements = None, posVec = None, velVec = None, t0 = 0, mass = 5.9722*1024):
        # Newtonian constant of gravitation
        self.gravitational_parameter = 6.67428e-11
        # Standard gravitational parameter
        self.mu = self.gravitational_parameter*mass
        print(self.mu)
        self.t0 = t0
        if not elements is None:
            self.a, self.i, self.e, self.LAN, self.omega, self.M0 = elements
        else:
            self.a, self.i, self.e, self.LAN, self.omega, self.M0 = self.get_orbital_elements(posVec, velVec)



    def get_orbital_elements(self, posVec, velVec):
        # Algorithm source: https://www.astro.uvic.ca/~tatum/celmechs/celm10.pdf
        AU = 1.495987e11 # m
        VelocityUnits = 29.7846917e3 # m/s

        def magnitude(vector):
            return np.sqrt(np.dot(vector, vector))

        def remove_ambiguity_sc(sinVal, cosVal):
            if cosVal >= 0 and sinVal >= 0:
                return np.arccos(cosVal)
            if cosVal >= 0 and sinVal <= 0:
                return np.arcsin(sinVal)
            if cosVal <= 0 and sinVal >= 0:
                return np.arccos(cosVal)
            if cosVal <= 0 and sinVal <= 0:
                return 2*np.pi - np.arccos(cosVal)
            
        def remove_ambiguity_st(sinVal, tanVal):
            if sinVal >= 0 and tanVal >= 0:
                return np.arcsin(sinVal)
            if sinVal >= 0 and tanVal <= 0:
                return np.pi - np.arcsin(sinVal)
            if sinVal <= 0 and tanVal >= 0:
                return np.pi + np.arctan(tanVal)
            if sinVal <= 0 and tanVal <= 0:
                return np.arcsin(sinVal)
        
        def argument_of_latitude(vector, inclination):
            xVec, yVec, zVec = vector
            vecMag = magnitude(vector)
            Beta_sin = zVec/vecMag
            Theta_sin = Beta_sin/np.sin(inclination)

            Lambda_sin = yVec/(xVec**2+yVec**2)**0.5
            Lambda_cos = xVec/(xVec**2+yVec**2)**0.5

            Lambda = remove_ambiguity_sc(Lambda_sin, Lambda_cos)
            print(Lambda)
            Theta_tan = np.tan(Lambda - LAN)/np.cos(inclination)

            return remove_ambiguity_st(Theta_sin, Theta_tan)

        posVec = np.array(posVec) / AU
        velVec = np.array(velVec)
        
        r = magnitude(posVec) # au
        V = magnitude(velVec)/VelocityUnits
        a = r/(2-r*V**2)

        # Vector noraml to orbit plane
        normalVec = np.cross(posVec, velVec)
        normalVec /= magnitude(normalVec)
        xNormal, yNormal, zNormal = normalVec
        i = np.arccos(zNormal)

        # To remove quadrant ambiguity
        print(normalVec)
        LAN_sin = xNormal/(xNormal**2+yNormal**2)**0.5
        LAN_cos = -yNormal/(xNormal**2+yNormal**2)**0.5

        if np.isnan(LAN_sin):
            LAN = 0
        else:
            LAN = remove_ambiguity_sc(LAN_sin, LAN_cos)

        

        theta = argument_of_latitude(posVec, i)
        psi = argument_of_latitude(velVec, i)

        if theta is None:
            theta = 90
        if psi is None:
            psi = 0
        
        l = r**2*V**2*np.sin(psi-theta)**2

        e = (1-l/a)**0.5

        true_anomaly = np.arccos(1/e*(l/r-1))

        # removing quadrant ambiguity
        if np.dot(posVec, velVec) < 0:
            true_anomaly = 2*np.pi - true_anomaly

        print("TRue" + str(true_anomaly*180/np.pi))
        omega = true_anomaly - theta


        eccentric_anomaly0 = np.arccos((e+np.cos(true_anomaly))/(1+e*np.cos(true_anomaly)))

        print(eccentric_anomaly0)
        if np.isnan(eccentric_anomaly0):
            eccentric_anomaly0 = 90

        if (eccentric_anomaly0 > 0) != (true_anomaly > 0):
            eccentric_anomaly0 *= -1

        M0 = eccentric_anomaly0 - e*np.sin(eccentric_anomaly0)

        #print((a, i*180/np.pi, e, LAN*180/np.pi, omega*180/np.pi, M0*180/np.pi))
        print((a*AU, i*180/np.pi, e, LAN*180/np.pi, omega*180/np.pi, M0*180/np.pi))

        return (a*AU, i, e, LAN, omega, M0)
    
    
    def calculateOrbit(self, t):
        # Algorithm source: https://downloads.rene-schwarz.com/download/M001-Keplerian_Orbit_Elements_to_Cartesian_State_Vectors.pdf
        # takes t in seconds to calculate mean anomaly M(t)
        tDelta = 86400*(t-self.t0)
        mean_anomaly = (self.M0 + tDelta*(self.mu/self.a**3)**0.5)%(2*np.pi)

        iterations = 15

        eccentric_anomaly = mean_anomaly

        

        for iteration in range(0, iterations):
            eccentric_anomaly = eccentric_anomaly - (eccentric_anomaly - self.e*np.sin(eccentric_anomaly) - mean_anomaly)/(1-self.e*np.cos(eccentric_anomaly))

        print(mean_anomaly)
        print(eccentric_anomaly)

        # True anomaly
        def arctan2(y, x):
            if x > 0:
                return np.arctan(y/x)
            elif x < 0 and y >= 0:
                return np.arctan(y/x) + np.pi
            elif x < 0 and y < 0:
                return np.arctan(y/x) - np.pi
            elif x == 0 and y > 0:
                return np.pi/2
            elif x == 0 and y < 0:
                return -np.pi/2
            else:
                return np.nan
            
        true_anomaly = 2*arctan2((1+self.e)**0.5*np.sin(eccentric_anomaly/2), (1-self.e)**0.5*np.sin(eccentric_anomaly/2))

        # Distance to central body
        rc = self.a*(1-self.e*np.cos(eccentric_anomaly))

        # Position vector in orbital frame
        o = rc * np.array([np.cos(true_anomaly), np.sin(true_anomaly)])

        # Velocity in orbital frame
        odot = (self.mu*self.a)**0.5/rc*np.array([-np.sin(eccentric_anomaly), (1-self.e**2)**0.5*np.cos(eccentric_anomaly)])

        def rotation_to_intertial(vector):
            x_orb, y_orb = vector[0], vector[1]
            x = (x_orb*(np.cos(self.omega)*np.cos(self.LAN) - np.sin(self.omega)*np.cos(self.i)*np.sin(self.LAN))
                - y_orb*(np.sin(self.omega)*np.cos(self.LAN) + np.cos(self.omega)*np.cos(self.i)*np.sin(self.omega)))
            y = (x_orb*(np.cos(self.omega)*np.sin(self.LAN) + np.sin(self.omega)*np.cos(self.i)*np.cos(self.LAN))
                - y_orb*(np.cos(self.omega)*np.cos(self.i)*np.cos(self.LAN) - np.sin(self.omega)*np.sin(self.LAN)))
            z = x_orb*(np.sin(self.omega)*np.sin(self.i)) + y_orb*(np.cos(self.omega)*np.sin(self.i))
            return np.array([x, y, z])

        return (rotation_to_intertial(o), rotation_to_intertial(odot))


In [35]:
#theOrbit = orbitElements(posVec = [(6378+500)*1000, 0, 0], velVec = [0, 4949, 4949])
AU = 1.495987e11
#theOrbit = orbitElements(posVec = [(6378+500)*1000, 0, 0], velVec = [0, 4949, 4949])
#theOrbit = orbitElements(posVec = [1.5*AU, 0.6*AU, 0.2*AU], velVec = [20000, 10000, 4000])
theOrbit = orbitElements(posVec = [224398050000, 89759220000, 29919740000], velVec = [20000, 10000, 4000], mass = 1.9884*10**30)

print(theOrbit.calculateOrbit(0))

1.3271138351999998e+20
[ 0.11026357 -0.55131785  0.82697677]
0.3805063771123646
0.46364760900080615
TRue174.67021404576099
1.624217147874925
(231241209781.5438, 34.21057985417024, 0.9951899675588027, 11.309932474020227, 162.04817801656273, 36.121944933230864)
0.6304468713089618
1.624217147874925
(array([ 2.36770823e+11,  4.48806337e+09, -5.47586434e+10]), array([22332.25532506, -1538.45115734, -3872.68731944]))


In [27]:
print(0.2*AU)

29919740000.0


In [ ]:

    def calculateOrbit(self, t):
        # takes t in seconds to calculate mean anomaly M(t)
        def M(self, t):
            tDelta = 86400*(t-self.t0)
            return (self.M0 + tDelta*(self.mu/self.a**3)**0.5)%(2*np.pi)

        # Eccentric anomaly
        def E(self, t):
            iterations = 15
            mean_anomaly = M(t)
            E0 = M(t)

            eccentric_anomaly = E0

            for iteration in range(0, iterations):
                eccentric_anomaly = eccentric_anomaly - (eccentric_anomaly - self.e*np.sin(eccentric_anomaly) - mean_anomaly)/(1-self.e*np.cos(eccentric_anomaly))
            
            return eccentric_anomaly

        # True anomaly
        def v(self, t):
            def arctan2(y, x):
                if x > 0:
                    return np.arctan(y/x)
                elif x < 0 and y >= 0:
                    return np.arctan(y/x) + np.pi
                elif x < 0 and y < 0:
                    return np.arctan(y/x) - np.pi
                elif x == 0 and y > 0:
                    return np.pi/2
                elif x == 0 and y < 0:
                    return -np.pi/2
                else:
                    return np.nan
                
            eccentric_anomoly = E(t)
            return 2*arctan2((1+self.e)**0.5*np.sin(eccentric_anomoly/2), (1-self.e)**0.5*np.sin(eccentric_anomoly/2))

        # Distance to central body
        def rc(self, t):
            return self.a*(1-self.e*np.cos(E(t)))

        # Position vector in orbital frame
        def o(self, t):
            true_anomaly = v(t) 
            return rc(t) * np.array([np.cos(true_anomaly), np.sin(true_anomaly)])

        # Veloc
        def oDot(self, t):
            eccentric_anomoly = E(t)
            return (self.mu*a)**0.5/rc(t)*np.array([-np.sin(eccentric_anomoly), (1-e**2)**0.5*np.cos(eccentric_anomoly)])

        def rotation_to_intertial(vector):
            x_orb, y_orb = vector[0], vector[1]
            x = (x_orb*(np.cos(omega)*np.cos(LAN) - np.sin(omega)*np.cos(i)*np.sin(LAN))
                - y_orb*(np.sin(omega)*np.cos(LAN) + np.cos(omega)*np.cos(i)*np.sin(omega)))
            y = (x_orb*(np.cos(omega)*np.sin(LAN) + np.sin(omega)*np.cos(i)*np.cos(LAN))
                - y_orb*(np.cos(omega)*np.cos(i)*np.cos(LAN) - np.sin(omega)*np.sin(LAN)))
            z = x_orb*(np.sin(omega)*np.sin(i)) + y_orb*(np.cos(omega)*np.sin(i))
            return np.array([x, y, z])

        def inertialPos(t):
            return rotation_to_intertial(o(t))

        def inertialVel(t):
            return rotation_to_intertial(oDot(t))